# DDoS dataset analysis

L'obiettivo di questo progetto è analizzare un dataset contenente informazioni relative sia ad attacchi DDoS che a normale traffico.
Questo per poter realizzare un'applicazione capace di distinguere il traffico sospetto da quello legittimo e poter quindi tempestivamente bloccare i tentativi di attacco.

## Descrizione del dataset

Il dataset è composto da una serie di campionamenti, ognuno dei quali fa riferimento ad una specifica connessione tra due macchine. 
Un campionamento, oltre ad identificare le due macchine e le interfacce di rete coinvolte nella comunicazione, contiene altre informazioni aggregate sullo stato della connessione stessa, come ad esempio il numero di pacchetti e di byte scambiati durante il periodo di campionamento.

Il dataset si può trovare a questo [link](https://www.kaggle.com/datasets/yahyaalhaj/biotddos).
Si compone di tre file, che possono essere scaricati come un unico file "zip" a questo [link](https://www.kaggle.com/datasets/yahyaalhaj/biotddos/download). 

### Descrizione dei file

Tutti e tre i file hanno lo stesso formato e contengono tre diverse categorie di attacchi, ovvero attacchi "TCP", "UDP" e "HTTP".
Non siamo interessati a tenere conto di questa distinzione, perciò consideriamo questi file come contenenti lo stesso tipo di dati.
Ogni file è in formato CSV e contiene un insieme di record, ognuno composto dai valori:
* "stime": la data e l'ora di inizio del campionamento in formato UNIX timestamp, dotato anche dei millisecondi 
* "flgs": le "flag" TCP e UDP osservate nei pacchetti che sono stati scambiati durante il campionamento
* "proto": il protocollo di livello di rete utilizzato dai pacchetti che sono stati scambiati durante il campionamento
* "saddr": l'indirizzo IP della sorgente della connessione che è interessata dal campionamento
* "sport": la porta della sorgente della connessione che è interessata dal campionamento
* "dir": la direzione del flusso dati
* "daddr": l'indirizzo IP della destinazione della connessione che è interessata dal campionamento
* "dport": la porta della destinazione della connessione che è interessata dal campionamento
* "pkts": il numero di pacchetti scambiati durante questo campionamento
* "bytes": il numero di byte scambiati durante questo campionamento
* "state": lo stato della connessione 
* srcid
* ltime
* seq
* dur
* mean
* stddev
* smac
* dmac
* sum
* min
* max
* soui
* doui
* sco
* dco
* spkts
* dpkts
* sbytes
* dbytes
* rate
* srate
* drate
* record
* attack
* category
* subcategory

In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":3, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
import java.net.InetAddress
import java.time.format.{ DateTimeFormatter, DateTimeParseException }
import java.time.{ Instant, LocalDateTime, ZoneId }
import scala.util.Try

// import NetworkProtocol.NetworkProtocol

case class Record(
    stime: LocalDateTime,
    protocol: String,
    saddr: String,
    sport: Long,
    dir: String,
    daddr: String,
    dport: Long,
    pkts: Long,
    bytes: Long,
    ltime: LocalDateTime,
    sbytes: Long,
    dbytes: Long,
    isDDoS: Boolean
)

object Record {

  def apply(r: Seq[String]): Option[Record] = (for {
      stime <- Try(Instant.ofEpochMilli((r.head.toDouble * 1000).toLong).atZone(ZoneId.systemDefault()).toLocalDateTime)
      protocol = r(2)
      saddr = r(3)
      sport <- Try(r(4).toLong)
      dir = r(5)
      daddr = r(6)
      dport <- Try(r(7).toLong)
      pkts <- Try(r(8).toLong)
      bytes <- Try(r(9).toLong)
      ltime <- Try(Instant.ofEpochMilli((r(12).toDouble * 1000).toLong).atZone(ZoneId.systemDefault()).toLocalDateTime)
      sbytes <- Try(r(28).toLong)
      dbytes <- Try(r(29).toLong)
      isDDoS <- Try(r(34).toInt).map(p => if (p == 1) true else false)
    } yield new Record(stime,protocol,saddr,sport,dir,daddr,dport,pkts,bytes,ltime,sbytes,dbytes,isDDoS)).toOption
}

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1655472000795_0005,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.net.InetAddress
import java.time.format.{DateTimeFormatter, DateTimeParseException}
import java.time.{Instant, LocalDateTime, ZoneId}
import scala.util.Try
defined class Record
defined object Record
Companions must be defined together; you may wish to use :paste mode for this.


In [3]:
val bucketName = "unibo-bd2122-nfarabegoli/ddos"

val pathTCPDataset = s"s3a://$bucketName/DDoS_TCP.csv"
val pathUDPDataset = s"s3a://$bucketName/DDoS_UDP.csv"
val pathHTTPDataset = s"s3a://$bucketName/DDoS_HTTP.csv"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketName: String = unibo-bd2122-nfarabegoli/ddos
pathTCPDataset: String = s3a://unibo-bd2122-nfarabegoli/ddos/DDoS_TCP.csv
pathUDPDataset: String = s3a://unibo-bd2122-nfarabegoli/ddos/DDoS_UDP.csv
pathHTTPDataset: String = s3a://unibo-bd2122-nfarabegoli/ddos/DDoS_HTTP.csv
res7: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1655472000795_0005/


In [4]:
val dataset = sc.textFile(s"$pathTCPDataset,$pathUDPDataset,$pathHTTPDataset")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dataset: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2122-nfarabegoli/ddos/DDoS_TCP.csv,s3a://unibo-bd2122-nfarabegoli/ddos/DDoS_UDP.csv,s3a://unibo-bd2122-nfarabegoli/ddos/DDoS_HTTP.csv MapPartitionsRDD[1] at textFile at <console>:35


In [5]:
dataset.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res8: Array[String] = Array("stime";"flgs";"proto";"saddr";"sport";"dir";"daddr";"dport";"pkts";"bytes";"state";"srcid";"ltime";"seq";"dur";"mean";"stddev";"smac";"dmac";"sum";"min";"max";"soui";"doui";"sco";"dco";"spkts";"dpkts";"sbytes";"dbytes";"rate";"srate";"drate";"record";"attack";"category";"subcategory")


In [6]:
val recordDataset = dataset.
    map(_.replace("\"", "")).
    map(_.split(";")).
    map(Record(_)).
    filter(_.isDefined).
    map(_.get).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

recordDataset: org.apache.spark.rdd.RDD[Record] = MapPartitionsRDD[6] at map at <console>:36


In [11]:
val datasetSize = recordDataset.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

datasetSize: Long = 38532513


## Query esplorative

### Percentuale di DDoS nel dataset


In [26]:
val ddosVolume = recordDataset.filter(_.isDDoS).count()
val legitVolume = recordDataset.filter(!_.isDDoS).count()
println(s"DDoS volume: $ddosVolume")
println(s"Legit volume: $legitVolume")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosVolume: Long = 38531238
legitVolume: Long = 1275
DDoS volume: 38531238
Legit volume: 1275


In [13]:
val ddosPercentage = ddosVolume / datasetSize.toDouble * 100

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosPercentage: Double = 99.99669110602778


### Conteggio dei flussi catturati nel dataset
Un flusso lo si identifica da (sourceIp, sourcePort, destinationIp, destinationPort)

In [20]:
val flowsDataset = recordDataset.
    groupBy(r => (r.saddr, r.sport, r.daddr, r.dport)).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

flowsDataset: org.apache.spark.rdd.RDD[((String, Long, String, Long), Iterable[Record])] = ShuffledRDD[13] at groupBy at <console>:30


In [21]:
val flowsCount = flowsDataset.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

flowsCount: Long = 1084088


In [27]:
val ddosFlowsCount = flowsDataset.filter { case (_, packets) => packets.forall(_.isDDoS) }.count()
val legitFlowsCount = flowsDataset.filter { case (_, packets) => packets.forall(!_.isDDoS) }.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ddosFlowsCount: Long = 1083994
legitFlowsCount: Long = 94


Determinati i flussi, possiamo calcolare mediamente quanti pacchetti sono scambiati per ogni flusso:

In [29]:
val meanPacketPerFlow = datasetSize / flowsCount.toDouble
val meanPacketDDoSFlow = ddosVolume / ddosFlowsCount.toDouble
val meanPacketLegitFlow = legitVolume / legitFlowsCount.toDouble

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

meanPacketPerFlow: Double = 35.54371324099151
meanPacketDDoSFlow: Double = 35.545619256195145
meanPacketLegitFlow: Double = 13.563829787234043
